In [96]:
user_of_interest = 'tnt'

# Cleaning Pipeline

In [97]:
from BeerMe.Pipeline import *

In [98]:
df = IMPORT_CLEAN_STEP(db_path='data/beer.db')



1. NA Count...
ABV                923
global_rating        0
user_rating       3882
IBU              47035
dtype: int64


2. Finding IQR outliers...
FEATURE ABV
num of outliers = 3,421
% of outliers = 3.33%


NA Count...
ABV              0
global_rating    0
user_rating      0
IBU              0
dtype: int64


In [99]:
df = df[~df.duplicated()]

In [100]:
df = df[['username', 'beer_name', 'beer_description', 'brewery', 'ABV', 'IBU', 'user_rating']]

In [101]:
df[df.duplicated()]

,username,beer_name,beer_description,brewery,ABV,IBU,user_rating
107169,tsharp93,Firestone Lager,Lager - Helles,Firestone Walker Brewing Company,4.5,17,3.50
107174,tsharp93,Head High,IPA - American,Kane Brewing Company,6.6,80,3.50
107176,tsharp93,The Drongo,Sour - Other,Clown Shoes,6.5,44.1594,4.25
107177,tsharp93,Forbidden Planet,IPA - Imperial / Double,Raduga,8.4,115,4.50
107178,tsharp93,They Both Melt,Lager - Munich Dunkel,Bluejacket,5.6,44.1594,3.25


# 0. Baseline - Average 

In [112]:
global_mean = np.mean(df['user_rating'])
print("Global user_rating mean = {:.2f}".format(global_mean))

Global user_rating mean = 3.71


In [113]:
beer_list = list(df[df['username']=='tsharp93']['beer_name'])

In [114]:
estimated_rating_list = []
error_list = []
for beer in beer_list:
    try:
        estimated_rating = global_mean
        estimated_rating_list.append(estimated_rating)

        user_rating = df[(df['username']=='tsharp93') & (df['beer_name']==beer)]['user_rating'].iloc[0].astype(float)
        error_list.append(estimated_rating-user_rating)
    except IndexError:
        print(beer)

In [115]:
mse = np.mean(np.array(error_list)**2)
mae = np.absolute(error_list).mean()
quarter_error_perc = 100 * np.sum(np.absolute(error_list) < 0.25) / len(error_list)
half_error_perc = 100 * np.sum(np.absolute(error_list) < 0.50) / len(error_list)

print("MSE = {:.2f}".format(mse))
print("MAE = {:.2f}".format(mae))
print("Errors within 0.25 = {:.2f} %".format(quarter_error_perc))
print("Errors within 0.50 = {:.2f} %".format(half_error_perc))

MSE = 0.23
MAE = 0.40
Errors within 0.25 = 35.00 %
Errors within 0.50 = 67.50 %


# 1. Cold Start

### For use when a user does not have enough data to build a model for his/her preferences

In [102]:
df = COSINE_STEP(df, user_of_interest)

User of Reference for Cosine Sim = tnt
(102598, 7)
(102598, 8)


In [103]:
beer_list = list(df[df['username']=='tsharp93']['beer_name'])

In [107]:
estimated_rating_list = []
error_list = []
for beer in beer_list:
    try:
        estimated_rating = df[ (df.sort_values('nearest_neighbor_rank')['beer_name'] == beer) & (df['username']!=user_of_interest) ]['user_rating'].iloc[0]
        estimated_rating_list.append(estimated_rating)

        user_rating = df[(df['username']=='tsharp93') & (df['beer_name']==beer)]['user_rating'].iloc[0].astype(float)
        error_list.append(estimated_rating-user_rating)
    except IndexError:
        print(beer)

In [109]:
mse = np.mean(np.array(error_list)**2)
mae = np.absolute(error_list).mean()
quarter_error_perc = 100 * np.sum(np.absolute(error_list) < 0.25) / len(error_list)
half_error_perc = 100 * np.sum(np.absolute(error_list) < 0.50) / len(error_list)

print("MSE = {:.2f}".format(mse))
print("MAE = {:.2f}".format(mae))
print("Errors within 0.25 = {:.2f} %".format(quarter_error_perc))
print("Errors within 0.50 = {:.2f} %".format(half_error_perc))

MSE = 0.33
MAE = 0.38
Errors within 0.25 = 42.50 %
Errors within 0.50 = 55.00 %


#### Problems with this approach: 
#### (1) Need another user to have the beer I have selected. If I want to get my user_rating for a beer that no one has tried, I can't do that. 
#### (2) Not very accurate. Almost half the points are outside of 0.5 error - sometimes worse than baseline

# 2. Content Based Filtering 

### Assumes the user of interest has enough data to build a model 

In [243]:
user_df = df[df['username'] == user_of_interest]
len(user_df)

1684

In [244]:
user_df.head()

,nearest_neighbor_rank,username,beer_name,beer_description,brewery,ABV,IBU,user_rating
100914,NaN,tnt,Society & Solitude #7,IPA - Imperial / Double,Hill Farmstead Brewery,8.0,44.1594,4.25
100915,NaN,tnt,Riwaka Single Hop Pale Ale,Pale Ale - American,Hill Farmstead Brewery,5.2,44.1594,5.00
100916,NaN,tnt,"Walk, Don't Run",Blonde Ale,Suarez Family Brewery,3.7,44.1594,4.50
100917,NaN,tnt,Czech Two,Pilsner - Czech,Gravely Brewing Co,4.1,39,4.25
100918,NaN,tnt,Spirit of Revolt,IPA - Imperial / Double,Revolution Brewing Company,8.2,52,4.00


#### Feature Engineering

In [245]:
user_df = convert_categorical(user_df, ['beer_description'])

In [246]:
user_df.drop(['username', 'nearest_neighbor_rank', 'beer_description', 'beer_name', 'brewery'], axis=1, inplace=True)

In [247]:
user_df.shape

(1684, 135)

In [248]:
user_df.head()

,ABV,IBU,user_rating,beer_description_Altbier,beer_description_American Wild Ale,beer_description_Barleywine - American,beer_description_Barleywine - English,beer_description_Belgian Blonde,beer_description_Belgian Dubbel,beer_description_Belgian Quadrupel,...,beer_description_Stout - Russian Imperial,beer_description_Strong Ale - American,beer_description_Strong Ale - English,beer_description_Table Beer,beer_description_Traditional Ale,beer_description_Wheat Beer - American Pale Wheat,beer_description_Wheat Beer - Other,beer_description_Wheat Wine,beer_description_Winter Ale,beer_description_Witbier
100914,8.0,44.1594,4.25,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100915,5.2,44.1594,5.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100916,3.7,44.1594,4.50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100917,4.1,39,4.25,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100918,8.2,52,4.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [249]:
features = user_df.columns[list(user_df.columns != 'user_rating')]
target = 'user_rating'

In [250]:
features

Index(['ABV', 'IBU', 'beer_description_Altbier',
       'beer_description_American Wild Ale',
       'beer_description_Barleywine - American',
       'beer_description_Barleywine - English',
       'beer_description_Belgian Blonde', 'beer_description_Belgian Dubbel',
       'beer_description_Belgian Quadrupel',
       'beer_description_Belgian Strong Dark Ale',
       ...
       'beer_description_Stout - Russian Imperial',
       'beer_description_Strong Ale - American',
       'beer_description_Strong Ale - English', 'beer_description_Table Beer',
       'beer_description_Traditional Ale',
       'beer_description_Wheat Beer - American Pale Wheat',
       'beer_description_Wheat Beer - Other', 'beer_description_Wheat Wine',
       'beer_description_Winter Ale', 'beer_description_Witbier'],
      dtype='object', length=134)

In [251]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(user_df[features], user_df[target], random_state=10)

In [252]:
from sklearn.linear_model import LassoCV
lassocv = LassoCV(fit_intercept=True, normalize=True, cv=5, random_state=10)
lassocv.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=None, normalize=True,
    positive=False, precompute='auto', random_state=10, selection='cyclic',
    tol=0.0001, verbose=False)

In [253]:
lassocv.alpha_

0.00030792048910093493

In [254]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=lassocv.alpha_,fit_intercept=True, normalize=True, random_state=10)
lasso.fit(X_train, y_train)

Lasso(alpha=0.00030792048910093493, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=True, positive=False, precompute=False,
   random_state=10, selection='cyclic', tol=0.0001, warm_start=False)

In [255]:
coef_dropped_perc = 100*np.sum(lasso.coef_ == 0) / len(lasso.coef_)
print("Percentage of estimators dropped = {:.2f} %".format(coef_dropped_perc))

Percentage of estimators dropped = 44.78 %


In [261]:
features_dropped = list(features[lasso.coef_ == 0])
print("Features dropped: ", features_dropped)

Features dropped:  ['beer_description_Altbier', 'beer_description_Barleywine - American', 'beer_description_Belgian Blonde', 'beer_description_Bière de Champagne / Bière Brut', 'beer_description_Bière de Garde', 'beer_description_Bock - Doppelbock', 'beer_description_Bock - Hell / Maibock / Lentebock', 'beer_description_Brown Ale - Belgian', 'beer_description_Brown Ale - English', 'beer_description_Brown Ale - Imperial / Double', 'beer_description_California Common', 'beer_description_Cider - Traditional', 'beer_description_Dunkelweizen', 'beer_description_English Bitter', 'beer_description_English Mild Ale', 'beer_description_Fruit Beer', 'beer_description_Golden Ale', 'beer_description_Gruit / Ancient Herbed Ale', 'beer_description_IPA - American', 'beer_description_IPA - Brut', 'beer_description_IPA - Imperial / Double', 'beer_description_IPA - International', 'beer_description_IPA - Milkshake', 'beer_description_Italian Grape Ale', 'beer_description_Kellerbier / Zwickelbier', 'beer

In [262]:
features_kept = list(features[lasso.coef_ != 0])
print("Features kept: ", features_kept)

Features kept:  ['ABV', 'IBU', 'beer_description_American Wild Ale', 'beer_description_Barleywine - English', 'beer_description_Belgian Dubbel', 'beer_description_Belgian Quadrupel', 'beer_description_Belgian Strong Dark Ale', 'beer_description_Belgian Strong Golden Ale', 'beer_description_Belgian Tripel', 'beer_description_Bière de Mars', 'beer_description_Blonde Ale', 'beer_description_Bock - Single / Traditional', 'beer_description_Brown Ale - American', 'beer_description_Cream Ale', 'beer_description_Dark Ale', 'beer_description_Extra Special / Strong Bitter', 'beer_description_Farmhouse Ale - Saison', 'beer_description_Grisette', 'beer_description_Hefeweizen', 'beer_description_IPA - Belgian', 'beer_description_IPA - Black / Cascadian Dark Ale', 'beer_description_IPA - English', 'beer_description_IPA - Imperial / Double Black', 'beer_description_IPA - Imperial / Double New England', 'beer_description_IPA - New England', 'beer_description_IPA - Red', 'beer_description_IPA - Rye', '

In [263]:
preds = lasso.predict(X_test)
error_list = preds - y_test

In [264]:
mse = np.mean(np.array(error_list)**2)
mae = np.absolute(error_list).mean()
quarter_error_perc = 100 * np.sum(np.absolute(error_list) < 0.25) / len(error_list)
half_error_perc = 100 * np.sum(np.absolute(error_list) < 0.50) / len(error_list)

print("MSE = {:.2f}".format(mse))
print("MAE = {:.2f}".format(mae))
print("Errors within 0.25 = {:.2f} %".format(quarter_error_perc))
print("Errors within 0.50 = {:.2f} %".format(half_error_perc))

MSE = 0.24
MAE = 0.36
Errors within 0.25 = 48.22 %
Errors within 0.50 = 76.96 %


### Improved Accuracy over both Cold Start and Baseline